In [5]:
import os
import numpy as np
np.random.seed(43)

In [230]:
setup_string = '''
# General setup
open morph-open-closed/open-closed.pdb
center
turn y 60

# Open bound structure get position of the ligand, missing in the morph
open 1ake
delete #1:.B
addh spec #1&protein hbond false
mmaker #0.1 #1

# Window appearance
windowsize 800 600
lighting reflectivity 0
set bg_color white
set silhouette
set silhouette_width 1.5

# Model appearance
color grey

# Add caption
# 2dlabel create title text "Binding and release of substrate leads to directional rotation \\n of amino acid side chains" xpos 0.1 ypos 0.92 color black

# Play movie sequence
# 2dlabel change title visibility show

# Hide everything
~ribbon; ~display
# Except ligand
display #1:AP5; color red #1:AP5; color byhet #1:AP5
'''

In [213]:
def pause_and_fade(pause_length, total_frames, file):
    file.write('# Fade out...\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format((pause + 1)* transparency_increment)
                  )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    file.write('~display #1:AP5\n')
    return total_frames

In [214]:
def pause_and_fade_in(pause_length, total_frames, file):
    file.write('# Fade in...\n')
    file.write('display #1:AP5\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format(100 - ((pause + 1)* transparency_increment))
        )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )

    return total_frames


def pause(pause_length, total_frames, file):
    for pause in np.arange(pause_length):
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    return total_frames

In [215]:
def add_bump(positive):
    sigma = 2
    if positive:
        mu = 6.0
    else:
        mu = -6.0
    rotation = sigma * np.random.randn(1) + mu
    return rotation[0]

In [216]:
def rotate_forward(models, residues, positives, 
                   list_of_previous_positions, file, colors, total_frames):
    #
    positions = np.empty(( len(residues), len(models) ))
    #
    for model in models:
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues): 
            
            if model == models[0]:

                positions[residue_index, model] = \
                                list_of_previous_positions[residue_index, model] + \
                                add_bump(positives[residue_index])
            else:
                positions[residue_index, model] = \
                                positions[residue_index, model - 1] + \
                                add_bump(positives[residue_index])

            additional_rotation = positions[residue_index, model] - \
                                  list_of_previous_positions[residue_index, model]

            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            file.write('rotation 2 {} 1; wait 1\n'.format(additional_rotation))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')


            print('Frame {:02d}: {:.2f} → {:.2f}, which is a rotation of additional {:.2f}'.format(model + 1, 
                                                               list_of_previous_positions[residue_index, model],
                                                               positions[residue_index, model],
                                                               additional_rotation))
            
        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
        )

    return positions, total_frames

In [279]:
def rotate_backward(models, residues, positives, list_of_previous_positions, file, colors, total_frames):
    #
    positions = np.zeros(( len(residues), len(models) + 1))
    #
    for model in models[::-1]:
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues):
            
            if model == models[-1]:

                positions[residue_index, model] = \
                                list_of_previous_positions[residue_index, model] + \
                                add_bump(positives[residue_index])
            else:
                positions[residue_index, model] = \
                                positions[residue_index, model + 1] + \
                                add_bump(positives[residue_index])
#                 print(f'{positions[residue_index, model]:.2f} = {positions[residue_index, model + 1]:.2f} + ' + \
#                       f'{add_bump(positives[residue_index]):.2f}')

            additional_rotation = positions[residue_index, model] - \
                                  list_of_previous_positions[residue_index, model]
            
            print('Frame {:02d}: {:.2f} → {:.2f}, which is a rotation of additional {:.2f}'.format(model + 1, 
                                                                               list_of_previous_positions[residue_index, model],
                                                                               positions[residue_index, model],
                                                                               additional_rotation))   
            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            file.write('rotation 2 {} 1; wait 1\n'.format(additional_rotation))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')


        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
        )
    
    return positions, total_frames

In [284]:
def gaussian(sigma, mu):
    rotation = sigma * np.random.randn(1) + mu
    return rotation[0]

def wiggle(frames, model, residue_list, total_frames, file):
    file.write('# Wiggle...\n')
    file.write('~display #1:AP5\n')

    file.write('ribbon #0.{}\n'.format(model))
    for frame in np.arange(frames):
        total_frames += 1
        for residue in residue_list:
            rotation = gaussian(10, -10)
            file.write('display #0.{}:{}\n'.format(model, residue))
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model, residue))
            file.write('rotation 2 {} 1; wait 1\n'.format(rotation))
            file.write('~rotation 2\n')

        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    return total_frames

def add_text(string, frames, total_frames, file):
    file.write('2dlabel create title text "{}" xpos 0.1 ypos 0.92 color black\n'.format(string))
    file.write('2dlabel change title visibility hide frames {}\n'.format(frames))
    for frame in np.arange(frames):
        total_frames += 1
        file.write('wait 1\n')
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
    )
    file.write('2dlabel delete title\n')
    return total_frames

In [288]:
residues = [20, 50, 120, 141, 200]
positives = [True, False, False, True, True]
pause_length = 20
morphs = np.arange(30)
total_frames = 0
colors = ['blue', 'green', 'purple', 'orange', 'yellow']

with open('morph_setup.cmd', 'w') as file:
    file.write(setup_string)
    total_frames = wiggle(10, '1', wiggle_residues, total_frames, file)

    #total_frames = add_text(string='When a substrate binds a protein...',
    #                    frames=10,
    #                    total_frames=total_frames,
    #                    file=file
    #                   )


    file.write('2dlabel create title text "When a substrate binds a protein..." xpos 0.1 ypos 0.92 color black\n')
    total_frames = pause_and_fade_in(5, total_frames, file)

    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives,
        list_of_previous_positions=np.zeros(( len(residues), len(morphs) + 1)),
        file=file, 
        colors=colors,
        total_frames=total_frames)
    
    file.write('2dlabel change title text "...there is a conformational change." xpos 0.1 ypos 0.92 color black\n')
    
    total_frames = pause_and_fade(5, total_frames, file)
    print('')
        
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        list_of_previous_positions=forward_rotations,
        colors=colors,
        file=file, 
        total_frames=total_frames)

    
    total_frames = pause_and_fade_in(5, total_frames, file)
    
    print('')
    file.write('2dlabel change title text "Cycling between conformations..." xpos 0.1 ypos 0.92 color black\n')

    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives, 
        list_of_previous_positions=backward_rotations,
        file=file, 
        colors=colors,
        total_frames=total_frames)


    total_frames = pause_and_fade(5, total_frames, file)
    print('')
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        list_of_previous_positions=forward_rotations,
        colors=colors,
        file=file, 
        total_frames=total_frames)
    
    file.write('2dlabel change title text "...leads to directional rotation of amino acid side chains." xpos 0.1 ypos 0.92 color black\n')

    total_frames = pause_and_fade_in(5, total_frames, file)
    print('')
    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives, 
        list_of_previous_positions=backward_rotations,
        file=file, 
        colors=colors,
        total_frames=total_frames)

    total_frames = pause_and_fade(5, total_frames, file)
    print('')
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        list_of_previous_positions=forward_rotations,
        colors=colors,
        file=file, 
        total_frames=total_frames)

    total_frames = pause(5, total_frames, file)

Frame 01: 0.00 → 4.31, which is a rotation of additional 4.31
Frame 01: 0.00 → -6.29, which is a rotation of additional -6.29
Frame 01: 0.00 → -3.84, which is a rotation of additional -3.84
Frame 01: 0.00 → 9.51, which is a rotation of additional 9.51
Frame 01: 0.00 → 8.89, which is a rotation of additional 8.89
Frame 02: 0.00 → 10.00, which is a rotation of additional 10.00
Frame 02: 0.00 → -14.04, which is a rotation of additional -14.04
Frame 02: 0.00 → -10.49, which is a rotation of additional -10.49
Frame 02: 0.00 → 16.55, which is a rotation of additional 16.55
Frame 02: 0.00 → 13.33, which is a rotation of additional 13.33
Frame 03: 0.00 → 16.72, which is a rotation of additional 16.72
Frame 03: 0.00 → -14.48, which is a rotation of additional -14.48
Frame 03: 0.00 → -14.15, which is a rotation of additional -14.15
Frame 03: 0.00 → 21.82, which is a rotation of additional 21.82
Frame 03: 0.00 → 18.85, which is a rotation of additional 18.85
Frame 04: 0.00 → 23.12, which is a rot

```
ffmpeg -r 10 -i open-closed-%03d.png -y -vf format=yuv420p morph.mp4
```

I probably need to create a function that goes forward, just starting with an initial value -- like I did for the backwards process, but without stepping through the frames backwards. In fact, I should just make this the general going forwards function and pass an array of zeros for the first trip forwards.

In [121]:
[[0] * len(residues), [0] * len(morphs)]

[[0], [0, 0, 0, 0, 0]]

Get sequence, add sequence index to `residues_list` if not GLY, PRO, or HIS.

In [239]:
fasta = 'MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG'

In [240]:
wiggle_residues = []
for index, residue in enumerate(fasta, 1):
    if residue is not 'G' and residue is not 'P' and residue is not 'H':
        wiggle_residues.append(index)

In [241]:
wiggle_residues

[1,
 2,
 3,
 4,
 5,
 6,
 8,
 11,
 13,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 26,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 82,
 83,
 84,
 86,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 124,
 125,
 127,
 129,
 131,
 132,
 133,
 135,
 136,
 137,
 138,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 149,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 176,
 178,
 179,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213]